In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from nltk import word_tokenize
from sklearn.feature_extraction import text
import urllib.request, json 

In [2]:
from sklearn.cluster import KMeans, MiniBatchKMeans, MeanShift
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

In [3]:
from langdetect import detect

In [19]:
from scipy.spatial.distance import cdist
import matplotlib.pylab as plt
from sklearn import metrics
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

## Define helper functions, initialize vectorizer

In [5]:
def list_to_string(df):    
    return ' '.join(df['text'])

In [6]:
def extract_language(df):    
    df['language'] = detect(df['text_concat'])
    return df

In [7]:
def extract_json(df):    
    df['course_id'] = df.values[0]['course_id']
    df['title'] = df.values[0]['title']
    df['text_concat'] = df.values[0]['description'] + ' ' + df.values[0]['subtitle'] + ' ' + df.values[0]['what_you_will_learn']
    df['url'] = df.values[0]['course_about_uri']
    df['language'] = detect(df['text_concat'])
    return df

In [204]:
# define stop words and add them to the standard english list

additional_stop_words = [
    'university',
    'college',
    'partner',
    'closely',
    'readings',
    'reading',
    'requires',
    'require',
    'title',
    'provider',
    'section',
    'video',
    'videos',
    'problem',
    'problems',
    'learners',
    'define',
    'able',
    'exam',
    'duke',
    'week',
    'teach',
    'learn',
    'goes',
    'description',
    'school',
    'illinois',
    'urbana',
    'duke',
    'champaign',
    'mit',
    'stanford',
    'harvard',
    'look',
    'null',
    'li',
    'em',
    'ul',
    'edx',
    'harvardx',
    'span',
    'href',
    '_blank',
    'mitx',
    'likely',
    'use',
    'like',
    'northwestern',
    'mooc',
    'enroll',
    'enrolling',
    'course',
    'certificate',
    'verified',
    'california',
    'san diego',
    'michigan',
    'll',
    'xseries',
    'micromasters',
    'actual',
    'actually',
    'teacher',
    'teaches',
    'edu',
    'educators',
    'work',
    'effort',
    'font',
    'acca',
    'nbsp',
    'br',
    'using',
    'strong',
    'stuff',
    'quot',
    'skills',
    'new',
    'understand'
]

stop_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

## edX
* * *

In [15]:
# with urllib.request.urlopen("https://www.edx.org/api/catalog/v2/courses/") as url:
#     edx_df = json.loads(url.read().decode())

edx_df = pd.read_json('../Documents/edx_courses.json')
edx_df = edx_df.apply(extract_json, axis=1)
edx_df = edx_df.set_index('title')

In [16]:
edx_df.head()

,items,course_id,text_concat,url,language
title,,,,,
Introduction to Marketing,"{'cid': 'v2_full_en:16781', 'current_language'...",course-v1:UBCx+Marketing1x+3T2015,<p>Marketing is a crucial function in all busi...,/course/introduction-marketing-ubcx-marketing1x,en
Introduction to Computer Science,"{'cid': 'v2_full_en:1022', 'current_language':...",course-v1:HarvardX+CS50+X,"<p>This is <strong>CS50x</strong>, Harvard Uni...",/course/introduction-computer-science-harvardx...,en
Basic Mandarin Chinese – Level 1,"{'cid': 'v2_full_en:19576', 'current_language'...",course-v1:MandarinX+MX101x+2T2016,<p>Introductory Mandarin is the first in a ser...,/course/basic-mandarin-chinese-level-1-mandari...,en
Accountant in Business,"{'cid': 'v2_full_en:33591', 'current_language'...",course-v1:ACCA+FAB-F1.x+3T2017,<p>Don’t have a business background but want t...,/course/accountant-business-acca-fab-f1-x-6,en
Android App Development for Beginners,"{'cid': 'v2_full_en:36566', 'current_language'...",course-v1:GalileoX+CAAD002X+2T2017,<p>This course is designed for students who ar...,/course/android-app-development-beginners-gali...,en


In [171]:
edx_df_english = edx_df[edx_df['language']=='en']
edx_df_english['provider'] = 'edx'

/Users/adleyhemphill/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [172]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [206]:
tf_vectorizer = CountVectorizer(stop_words = stop_words, 
                             ngram_range = (1,3),
                                token_pattern='(?u)\\b[a-zA-z][a-zA-z]+\\b',
                            max_df=0.95, min_df=2)

tfidf_vectorizer = TfidfVectorizer(stop_words = stop_words, 
                             ngram_range = (1,3),
                                   token_pattern='(?u)\\b[a-zA-z][a-zA-z]+\\b',
                            max_df=0.95, min_df=2)

In [207]:
tf = tf_vectorizer.fit_transform(edx_df_english.text_concat)
tfidf = tfidf_vectorizer.fit_transform(edx_df_english.text_concat)

In [208]:

lda = LatentDirichletAllocation(n_topics=50)
lda.fit(tf)

tfidf_vectorizer._validate_vocabulary()
tf_feature_names = tf_vectorizer.get_feature_names()


/Users/adleyhemphill/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [201]:
print_top_words(lda, tf_feature_names, 10)

Topic #0:
web, android, microsoft, application, development, app, mobile, server, azure, windows
Topic #1:
writers, novel, patient, writing, thinking, cancer, disease, creative, art, prevention
Topic #2:
energy, solar, cells, technologies, power, systems, photovoltaic, engineering, solar energy, design
Topic #3:
physics, quantum, mechanics, org, atomic, amp, optical, www, www org, https
Topic #4:
urinary, respiratory, answer questions data, available statistical software, techniques answer, statistical techniques answer, questions data, data widely available, available statistical, basic statistical techniques
Topic #5:
newtonian, newtonian mechanics, assigned, orbits, planetary, unified overview, planetary orbits, multi concept, newtonian mechanics year, year introductory
Topic #6:
business, water, license, design, delft, model, cc, sa, commons, creative
Topic #7:
organic, semiconductors, materials, chemistry, devices, electronic, properties, structure, transport, models
Topic #8:
soc

In [202]:
nmf = NMF(n_components=20, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)


print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 10)


Topics in NMF model:
Topic #0:
design, business, social, development, learning, world, systems, technology, global, research
Topic #1:
link, contact, nofollow target follow, target follow link, target follow, class external link, external link, class external, follow link, nofollow target
Topic #2:
data abstraction, java, abstraction, programming, data, formal solutions, oriented framework, object oriented framework, object oriented, integrated development
Topic #3:
ap, ap physics, physics, prepare ap, ap biology, prepare, biology, advanced placement, placement, placement ap
Topic #4:
data sheets, sheets, sans, text, read discuss write, vocabulary language support, vocabulary language, understanding text, discuss write text, write text
Topic #5:
data, big data, big, analytics, analysis, data analysis, data science, statistical, machine learning, machine
Topic #6:
project, project management, management, projects, organizational, project manager, manager, capstone, closing, initiation


## trying pyldavis

In [198]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [212]:
pyLDAvis.sklearn.prepare(lda, tf ,tf_vectorizer)

/Users/adleyhemphill/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
22     41.720707        1       1 -0.187297  0.123916
28     10.835896        1       2 -0.296116 -0.040228
29      5.672669        1       3 -0.286538 -0.073120
46      4.729829        1       4 -0.204337 -0.207166
43      3.582681        1       5 -0.224424  0.004949
14      3.439028        1       6 -0.163375 -0.064198
30      3.038805        1       7 -0.133446  0.289204
19      2.491385        1       8 -0.167967 -0.074979
45      2.218833        1       9 -0.145790  0.069935
6       2.084918        1      10 -0.094583 -0.088687
31      2.081498        1      11 -0.113412  0.079016
12      1.695880        1      12 -0.077067  0.104424
11      1.314668        1      13 -0.032111 -0.048047
9       1.138573        1      14 -0.052670 -0.036168
47      1.004966        1      15 -0.023716  0.049186
39      0.946994        1      16  0.007465  0.005631
0       0.853017        1      17  0.005751 -0.015262
34      0.815393        1      18 -0.009041  0.165826
15      0.765960        1      19  0.013219  0.037324
3       0.745311        1      20  0.015623 -0.018933
44      0.727953        1      21  0.026395  0.048841
21      0.690254        1      22  0.003897 -0.014919
32      0.651491        1      23  0.033184 -0.020225
4       0.618303        1      24  0.027552 -0.004897
48      0.607231        1      25  0.043862 -0.004936
10      0.595109        1      26 -0.004265 -0.038157
27      0.501212        1      27  0.046356 -0.007859
24      0.463107        1      28  0.042308 -0.030718
8       0.441115        1      29  0.073985 -0.020596
33      0.419986        1      30  0.067298 -0.006683
20      0.396717        1      31  0.077616 -0.015226
35      0.393153        1      32  0.079141 -0.023458
2       0.371759        1      33  0.067772 -0.023687
36      0.297530        1      34  0.094360 -0.016923
13      0.288258        1      35  0.082593 -0.006540
41      0.268308        1      36  0.098019 -0.017654
16      0.229979        1      37  0.097261 -0.010703
18      0.190190        1      38  0.095315 -0.007177
40      0.181893        1      39  0.096105 -0.002223
17      0.156274        1      40  0.097333 -0.010954
23      0.109660        1      41  0.101976 -0.008975
7       0.089840        1      42  0.100856 -0.006536
5       0.055302        1      43  0.095664 -0.002540
42      0.018720        1      44  0.089703 -0.002541
49      0.012672        1      45  0.090175 -0.001405
1       0.012226        1      46  0.089192 -0.001117
25      0.010752        1      47  0.089552 -0.000921
38      0.007998        1      48  0.088863 -0.001464
26      0.007998        1      49  0.088809 -0.001031
37      0.007998        1      50  0.088954 -0.001398, topic_info=      Category         Freq                                  Term  \
term                                                                
8379   Default  1744.000000                                  data   
19229  Default   729.000000                              learning   
9063   Default   707.000000                                design   
28288  Default   679.000000                              research   
30507  Default   395.000000                                social   
30130  Default   605.000000                                sheets   
8552   Default   601.000000                           data sheets   
1481   Default   546.000000                                    ap   
9500   Default   555.000000                           development   
31852  Default   763.000000                              students   
32642  Default   515.000000                               systems   
15271  Default   493.000000                                health   
26500  Default   500.000000                               project   
4117   Default   556.000000                              business   
17058  Default   393.000000                  